In [ ]:
import os
import re
import pickle
from os import path
import pandas as pd
import numpy as np
import json
from collections import Counter
import matplotlib.pyplot as plt
from scipy import stats

# Other analysis

## tf-idf on the modifiers and possessions

In [ ]:
power = pd.read_csv("/content/output_9.csv")

In [ ]:
power["genre"] = power["genre"].apply(tolist)
power = power.join(pd.get_dummies(power.genre.explode()).sum(level=0))
power

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


,Unnamed: 0,name,gender,power1,power2,power3,power4,power5,series_name,volume_name,...,Horror,Josei,Mature,Mystery,Psychological,Romance,Sci-Fi,Shoujo,Slice of Life,Supernatural
0,0,Huey,he/him/his,0.1810344827586207,0.20930232558139536,0.24390243902439024,0.3036649214659686,-0.15384615384615385,Dantalian_no_Shoka,Volume1,...,1,0,0,1,0,0,0,0,0,1
1,1,Dalian,she/her,0.16176470588235295,0.22727272727272727,0.11403508771929824,0.3305785123966942,0.42857142857142855,Dantalian_no_Shoka,Volume1,...,1,0,0,1,0,0,0,0,0,1
2,2,Hal,he/him/his,/,/,/,/,0.2866666666666667,Dantalian_no_Shoka,Volume1,...,1,0,0,1,0,0,0,0,0,1
3,3,Roy,he/him/his,/,0.3434343434343434,0.3,/,/,Dantalian_no_Shoka,Volume1,...,1,0,0,1,0,0,0,0,0,1
4,4,Mabel,she/her,/,/,/,/,0.19834710743801653,Dantalian_no_Shoka,Volume1,...,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,2528,Yawakaze,she/her,/,/,0.10714285714285714,0.6666666666666666,0.10526315789473684,"Ore_no_Nounai_Sentakushi_ga,_Gakuen_Love_Come_...",Volume1,...,0,0,0,0,0,1,0,0,1,0
2529,2529,Amakusa,she/her,0.2,-0.0625,0.2692307692307692,0.0,0.0,"Ore_no_Nounai_Sentakushi_ga,_Gakuen_Love_Come_...",Volume1,...,0,0,0,0,0,1,0,0,1,0
2530,2530,Konacchi,she/her,/,/,/,0.5714285714285714,0.0,"Ore_no_Nounai_Sentakushi_ga,_Gakuen_Love_Come_...",Volume1,...,0,0,0,0,0,1,0,0,1,0
2531,2531,Amacchi,she/her,/,/,/,-0.3076923076923077,-0.3,"Ore_no_Nounai_Sentakushi_ga,_Gakuen_Love_Come_...",Volume1,...,0,0,0,0,0,1,0,0,1,0


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import ast

In [ ]:
cv=CountVectorizer(stop_words='english')
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)

In [ ]:
def get_tfidf(doc):
  # doc: a list of lists, each sublist is a collection of words
  tfidf_vectorizer=TfidfVectorizer(stop_words='english', use_idf=True,sublinear_tf=True,smooth_idf=True)
  tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(doc)
  tfidf_df = pd.DataFrame(tfidf_vectorizer_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names())
  return tfidf_df


In [ ]:
def get_doc(df,mod_poss,genre=None):
  # return a list with the first element being a string of words for males and the second element that for females
  # mod_poss: "mod" for modifiers; "poss" for possessions

  if genre is not None:
    df = df[df[genre]==1]
  male = df[df["gender"]=="he/him/his"]
  female = df[df["gender"]=="she/her"]

  male_list = []
  for i in male[mod_poss].tolist():
    if i!="{}":
      mydict = ast.literal_eval(i)
      for key,item in mydict.items():
        male_list.extend([key]*item)

  female_list = []
  for i in female[mod_poss].tolist():
    if i!="{}":
      mydict = ast.literal_eval(i)
      for key,item in mydict.items():
        female_list.extend([key]*item)

  mydoc = [" ".join(male_list)," ".join(female_list)]
  return mydoc

In [ ]:
def get_top(df,mod_poss,genre=None):
  # return the top 20 words with highest tf-idf scores for both male and female
  
  mydoc = get_doc(df,mod_poss,genre)

  tf_idf_vector = get_tfidf(mydoc)
  male = pd.DataFrame(tf_idf_vector.iloc[0,:]).sort_values(by=[0],ascending=False).reset_index()
  male.rename({'index': mod_poss, 0: 'tf-idf'}, axis=1, inplace=True)
  female = pd.DataFrame(tf_idf_vector.iloc[1,:]).sort_values(by=[1],ascending=False).reset_index()
  female.rename({'index': mod_poss, 1: 'tf-idf'}, axis=1, inplace=True)

  return male.head(20).reset_index(),female.head(20).reset_index()


### take a look at the possessions for female and male characters in the whole corpus (do not group by genre)

In [ ]:
m,f = get_top(power,"poss")

In [ ]:
female = list(f["poss"])
male = list(m["poss"])
pd.DataFrame({"female":female,"male":male})

,female,male
0,eye,head
1,body,eye
2,head,body
3,face,face
4,hand,hand
5,voice,heart
6,mouth,voice
7,heart,mind
8,hair,word
9,word,mouth


### now take genres into consideration - create a doc groupped by genres and gender

In [ ]:
all_doc = []
genre_gender = []
for genre in list(set(power.genre.explode())):
  mydoc = get_doc(power,"mod",genre)
  all_doc.extend(mydoc)
  genre_gender.extend([f"{genre}_male",f"{genre}_female"])

In [ ]:
tf_idf_vector = get_tfidf(all_doc)
tf_idf_vector.insert(0,"genre_gender",genre_gender)
tf_idf_vector = tf_idf_vector.set_index('genre_gender')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def view_top(df,genre,gender):
  # view the top 20 words with high tf-idf for a given genre and gender
  df = pd.DataFrame(df.loc[f"{genre}_{gender}"]).reset_index()
  df.rename(columns = {'index':'word'}, inplace = True)
  df = df.sort_values(f"{genre}_{gender}",ascending=False).head(20).reset_index()
  return df

In [ ]:
def compare(df,genre):
  # put the top 20 major words for male and female characters in a given genre into a dataframe for better comparision
  
  female = list(view_top(df,genre,"female").word)
  male = list(view_top(df,genre,"male").word)
  return pd.DataFrame({"female":female,"male":male})

In [ ]:
compare(tf_idf_vector,"Shoujo")

,female,male
0,psychic,exorcist
1,medium,okay
2,positive,assistant
3,pretty,hunter
4,okay,able
5,able,monk
6,professional,thoughtful
7,dead,incredible
8,fine,uncomfortable
9,lonely,difficult


## try word embedding

In [ ]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

In [ ]:
!unzip /content/glove.6B.100d.txt.zip

Archive:  /content/glove.6B.100d.txt.zip
  inflating: glove.6B.100d.txt       
  inflating: __MACOSX/._glove.6B.100d.txt  


In [ ]:
# read embeddings from glove
embeddings_dict = {}
with open("/content/glove.6B.100d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [ ]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

In [ ]:
def get_difference(tfidf_female,tfidf_male):
  # return the top 3 closest word to the distance between the top words for male and top words for female
  
  emb = 0
  for i in tfidf_female.word:
    emb = emb + embeddings_dict.get(i,0)
  for i in tfidf_male.word:
    emb = emb - embeddings_dict.get(i,0)
  return find_closest_embeddings(emb)[:3]

In [ ]:
a = view_top(tf_idf_vector,"Josei","female").reset_index()
b = view_top(tf_idf_vector,"Josei","male").reset_index()

In [ ]:
get_difference(a,b)